This tutorial will show you how to load and manipulate files in Python. The key take-away here is the power of scripting. By scripting the file processing, you can batch import many files at once, you can clean-up and add values to your data, and you can repeat the task later when you get new files.

We are going to process a large file from the U.S. Bureau of Labor Statistics, the 2016 annual report on employment for the entire U.S., and all U.S. states and counties, broken down by industry and ownership sector (public employers and private employers).

Our goal is to loop through the rows in this large file (500 MB unzipped) and filter only the rows with data for the U.S., States and counties -- we are going to skip data for metropolitan areas and other niche geographies. 

We are also going to filter the detail level - the source data breaks industries down six ways, depending on how detailed we want to get -- for our purposes, the grand total and the mid-level detail (three-digit industry codes) is fine.

Because the industry and geographic area are provided to us as codes, we are going to create a lookup function that will insert columns contained the name of the industry and the geography. 

And we are also going to create a function that will attach an 'inflator' in case we want to adjust the dollar amounts for inflation -- in this case, going from 2016 to 2017 isn't such a big deal, remember, this script could be used on other years, and the more you go back in the past, the more inflation is a factor.

To start with, let's create the look-up functions that we will later call when we're doing the actuall file processing.

create a lookup for the industry title
 download this file 
https://raw.githubusercontent.com/gebelo/training2018/master/industry_titles.csv 
and save it locally... 
in theory, we could have done this step also in Python but doing it this way 
allows you to peruse the file before we work with it.


In [1]:
import csv
# we're going to open the file and turn it into a Python object 
infile=open("industry_titles.csv") #CHANGE THIS TO THE PATH TO YOUR FILE
# we're going to then convert the object into a Python CSV object, which will give us access to the CSV library of commands
reader=csv.reader(infile)
reader

In [3]:
# we're going to start at the top of the file and declare the first row to be the headers
headers=next(reader)
headers

['industry_code', 'industry_title']

In [4]:
# we're going to create a Python dictionary -- a key, and a value -- that will be our lookup.
lk_naics={}
#we're going to loop through the remaining rows in the file and populate our dictionary
#as we go through the row, the CSV reader is parsing each into two parts, row[0] and row[1]
for row in reader:
    #what we're doing here is populating lk_naics and setting the first value, row[0] to equal the second value row[1] - but we're getting rid
    #of the string 'NAICS' and the code itself, which would be redundant. And finally, we're getting rid of any
    #extra spaces
    lk_naics[row[0]]=row[1].replace('NAICS','').replace(row[0],'').strip()
#when we're done, we close the file to cleanup memory    
infile.close

<function TextIOWrapper.close>

In [6]:
#these are the kind of lookups we can use when we do the main data import...
print(lk_naics["562119"])
print(lk_naics["111110"])

Other waste collection
Soybean farming


In [7]:
#the entire function
# we're going to use the Python csv library
import csv
# we're going to open the file and turn it into a Python object 
infile=open("industry_titles.csv")
# we're going to then convert the object into a Python CSV object, which will give us access to the CSV library of commands
reader=csv.reader(infile)
# we're going to start at the top of the file and declare the first row to be the headers
headers=next(reader)
# we're going to create a Python dictionary -- a key, and a value -- that will be our lookup.
lk_naics={}
#we're going to loop through the remaining rows in the file and populate our dictionary
#as we go through the row, the CSV reader is parsing each into two parts, row[0] and row[1]
for row in reader:
    #what we're doing here is populating lk_naics and setting the first value, row[0] to equal the second value row[1] - but we're getting rid
    #of the string 'NAICS' and the code itself, which would be redundant. And finally, we're getting rid of any
    #extra spaces
    lk_naics[row[0]]=row[1].replace('NAICS','').replace(row[0],'').strip()
#when we're done, we close the file to cleanup memory    
infile.close

<function TextIOWrapper.close>

In [8]:
#the area titles function is more straightforward
#download the file from here https://raw.githubusercontent.com/gebelo/training2018/master/area_titles.csv and
#save it locally

infile=open("area_titles.csv") #CHANGE TO YOUR PATH
reader=csv.reader(infile)
headers=next(reader)
lk_area={}
for row in reader:
    lk_area[row[0]]=row[1]
    
infile.close

print(lk_area["34003"])

#let's discuss briefly why this worked....

Bergen County, New Jersey


In [11]:
#and lastly, our inflation table
#grab it from here https://raw.githubusercontent.com/gebelo/training2018/master/inflation.csv
#note that there are different inflation indexes in here, we're using the one called "inflator"
#which is column index number 2 -- the third column when we start counting with zero

infile=open("inflation.csv") #change to your path
reader=csv.reader(infile)
headers=next(reader)
lk_inflation={}
for row in reader:
    lk_inflation[row[0]]=row[2]


infile.close
print(lk_inflation["2016"])
#any questions on why this worked?

1.0213011


 and now for the exciting conclusion of this exercise... let's grab the giant data file 
from the bls web site  -- https://data.bls.gov/cew/data/files/2016/csv/2016_annual_singlefile.zip
unzip the file locally on your computer.



let's look at some file layouts and code sheet to see what we're doing

the file layout is here:
https://data.bls.gov/cew/doc/layouts/csv_annual_layout.htm

it looks like the key variables are columns 1 through 16 -- this means we're only going to import
python index columns 0 through 15, and then attach our lookup values.

to filter for just the total and three-digit industry codes, we refer to the table here:
https://data.bls.gov/cew/doc/titles/agglevel/agglevel_titles.htm

for the us, we want levels 10 and 15; for states, 50 and 55; for counties, we want 70 and 75



In [12]:
# create a variable with the path to the file
f='2016.annual.singlefile.csv' #CHANGE TO YOUR PATH
f

'2016.annual.singlefile.csv'

In [13]:
#create new file with the prefix 'new_' attached to the current file's name
file_out = open("new_2016.annual.singlefile.csv", 'w') #change to your path

In [14]:
#open original file    
o = open(f)
#create a Python csv object called 'file_in'
file_in = csv.reader((o), delimiter=",")
#create a Python object that will write to 'file-out'
csv_writer = csv.writer(file_out)

In [15]:
#insert headers from original file into the new file, making room for 3 new columns based on our lookups
csv_writer.writerow(next(file_in)+["area_label","industry_label","inflator"])
#loop through the original file, lookup labels and append them to row    
for row in file_in:
        #run our lookups and store reponse in variables
        
        the_area=lk_area[row[0]] # this runs 'area_fips' through our lk_area dictionary
        the_industry=lk_naics[row[2]] # this runs 'industry_code' through our lk_naics dictionary
        the_inflator=lk_inflation[row[5]] # this runs the year for our data through our inflation dictionary
        row=row[0:15]
        row.append(the_area)
        row.append(the_industry)
        row.append(the_inflator)
        
#write the row with the appendages to the new file only if it's the grand total or a three-digit industry
        if row[3] in ("10","15","50","55","70","75"):
                csv_writer.writerow(row)

file_out.close()
o.close()

#we've gone from 500 mbs to 35 in about 30  seconds....

Let's do one more example -- this time we're going to do some processing on a tab-delimited file. But in addition, we're going to use some string functions to further split apart one of the fields into components.

We're going to use Python to grab the file directly from the Web using the 'requests' library.

And finally, we're going to briefly introduce the Pandas library to peak at our output CSV.

This file is also from the U.S. Bureau of Labor Statistics, the Local Area Unemployment program.

The file layout is here
https://download.bls.gov/pub/time.series/la/la.txt

The key section:

Field #/Data Element	Length		Value (Example)		

1. series_id		  20		LASBS060000000000003          

2. year			   4		1976

3. period		   3		M01

4. value		  12      	9.4
				 
5. footnote_codes	  10		It varies
				

The series_id (LASBS060000000000003) can be broken out into:

Code					Value

survey abbreviation	=		LA
seasonal		=		S
area_code		=		BS0600000000000
measure_code		=		03




In [16]:
#let's grab the file and save it locally
import requests  # the lib that handles the url stuff

r = requests.get("https://download.bls.gov/pub/time.series/la/la.data.0.CurrentU15-19")  
with open(r'lau.txt', 'wb') as f: #change to your path
    f.write(r.content)

In [17]:
# how many lines are in there? 
text_file = open("lau.txt", "r") #change to your path
lines = text_file.readlines()
print(len(lines))
text_file.close()

1318605


In [19]:
#let's convert this into a csv file
import csv
#create a csv file
mycsv = csv.writer(open('lau.csv', 'w')) #change to your path
#write out the headers -- note we're going to split the original series id into components areacode and measurecode
mycsv.writerow(['seriesid', 'areacode','measurecode','year','period','value'])
#loop through the text file skipping the first row, which has the original headers
for line in lines[1:len(lines)]:
                #for each row -- split it using the tabs, the first piece is the series id
                 seriesid=line.split("\t")[0]
                #within seriesid, characters 3 through 18 are the area id
                 areacode=line.split("\t")[0][3:18]
                #with the seriesid, characters 18 through 20 are the measurecode
                 measurecode=line.split("\t")[0][18:20]
                #year, period and value are tabs 1,2 and 3    
                 year=line.split("\t")[1]
                 period=line.split("\t")[2]
                 value=line.split("\t")[3]
                #write the row
                 mycsv.writerow([seriesid, areacode,measurecode,year,period,value])

In [30]:
#lets open the csv with the Pandas library to see what it looks like
import pandas as pd
#this command opens the csv file and assigns datatypes to the columns, string 20, string 15, etc
laus = pd.read_csv('lau.csv',encoding='ascii',dtype={'seriesid': 'S20', 'areacode':'S15', 'measurecode':'S2', 'period':'S3'})
laus.head()
#this is the Python equivalent of a spreadsheet or database table -- you'll learn more about working with these
#in later classes

,seriesid,areacode,measurecode,year,period,value
0,b'LAUBS060000000000003',b'BS0600000000000',b'03',2015,b'M01',7.0
1,b'LAUBS060000000000003',b'BS0600000000000',b'03',2015,b'M02',6.7
2,b'LAUBS060000000000003',b'BS0600000000000',b'03',2015,b'M03',6.6
3,b'LAUBS060000000000003',b'BS0600000000000',b'03',2015,b'M04',6.0
4,b'LAUBS060000000000003',b'BS0600000000000',b'03',2015,b'M05',5.9
